In [139]:
import pandas as pd
import random

In [140]:
trans_engine = ['youdao', 'google', 'opus']
tl_set = ['Chinese', 'Italian']
data = {'Chinese': {}, 'Italian':{}}
for tl in tl_set:
    for engine in trans_engine:
        test_meta = pd.read_csv('data/MOH-X/'+tl+'/'+engine+'/test_liter.tsv', sep='\t')
        test_liter = pd.read_csv('data/MOH-X/'+tl+'/'+engine+'/train_liter.tsv', sep='\t')
        data[tl][engine] = [test_meta, test_liter]
data['Chinese']['opus'][0]

,target,index,en,cn,POS
0,ruined,2,You have ruined my car by pouring sugar in the...,<pad> 你把糖倒进罐子里 毁了我的车!,VERB
1,tack,0,tack the notice on the board.,<pad> 把通知写在黑板上,VERB
2,crawl.,12,European children learn the breast stroke; the...,<pad> 欧洲儿童学会了乳房中风;他们往往不会爬行。,VERB
3,guard,1,Please guard my possessions while I'm away.,<pad> 请在我离开的时候保护我的财产,VERB
4,spread.,2,the infection spread.,<pad> 感染传播。,VERB
5,volleyed,2,Gun shots volleyed at the attackers.,<pad> 向袭击者开枪。,VERB
6,exploded,1,We exploded the nuclear bomb.,<pad> 我们引爆了核弹,VERB
7,showered,1,Meteors showered down over half of Australia.,<pad> 超过半个澳洲的半个地区 气象都冲下大雨,VERB
8,swims,2,this snake swims through the soil where it lives.,<pad> 这种蛇在它居住的土壤中游荡,VERB
9,nags,1,She nags her husband all day long.,<pad> 她整天都缠着她丈夫,VERB


In [145]:
import jieba
import re
def extract_trans(data, pad, t):
    trans = []
    for s in data[t]:
        if pad:
            _ = re.search('<pad> (.*)', s)
            s = _[1]
        if t == 'cn':
            s = jieba.lcut(s)
            s = ' '.join(s)
        trans.append(s)
    return trans
        
def extract_source(dataset):
    en = []
    label = []
    index = []
    m = 1
    for set_ in [0,1]:
        data = dataset[set_]
        for i, s in enumerate(data['en']):
            _ = s.split(' ')
            _[int(data['index'][i])] = '@'+_[int(data['index'][i])]+'@'
            s = ' '.join(_)
            en.append(s)
            label.append(0)
            index.append(data['index'][i])
        m = 0
    return label, index, en
    
    

In [146]:
def save_raw_data(data, tl):
    data_dir = 'data/MOH-X/'+tl+'/raw_liter_data.csv'
    raw_data = pd.DataFrame(data)
    raw_data.to_csv(data_dir)
    return raw_data

def save_doccano(data, tl):
    text = []
    label = []
    index = []
    r = list(range(len(data)))
    random.shuffle(r)
    for i in r:
        index.append(i)
        en = data.loc[i]['en']+'\r\n'+'youdao: '+data.loc[i]['youdao']\
        +'\r\n'+'google: '+data.loc[i]['google']\
        +'\r\n'+'opus: '+data.loc[i]['opus']
        text.append(en)
        '''
        zh = 'opus: '+data.loc[i]['youdao']+' Equivalence: . Emotion: . Fluency: . Intelligibility: . Fidelity: . Authentic: .'\
        +'\r\n'+'youdao: '+data.loc[i]['google']+' Equivalence: . Emotion: . Fluency: . Intelligibility: . Fidelity: . Authentic: .'\
        +'\r\n'+'google:'+data.loc[i]['opus']+' Equivalence: . Emotion: . Fluency: . Intelligibility: . Fidelity: . Authentic: .'\
        +'\r\n'+'reference:. Equivalence: '
        '''
        zh = ''
        label.append(zh)
    
    doccano = pd.DataFrame({'index': index,'text':text, 'label':label, 'metaphorical':data['label']})
    doccano.to_csv('data/MOH-X/'+tl+'/doccano_literal.csv')
    

In [147]:
for tl in tl_set:
    mix_data = {}
    if tl == 'Chinese':
        t = 'cn'
    elif tl == 'Italian':
        t = 'it'
    
    label, index, en = extract_source(data[tl]['google'])
    mix_data['label'] = label
    mix_data['index'] = index
    mix_data['en'] = en
    for engine in trans_engine:
        pad = False
        if engine == 'opus':
            pad = True
        meta = extract_trans(data[tl][engine][0], pad, t)
        liter = extract_trans(data[tl][engine][1], pad, t)
        mix_data[engine] = meta+liter
    doccano_data = save_raw_data(mix_data, tl)
    save_doccano(doccano_data, tl)

In [109]:
print(random.randrange(1,100))
for i in random.randrange(1,100,100):
    print(i)


77


TypeError: 'int' object is not iterable